# Facial Anti-Spoofing using Deep Neural Network Approaches

In [102]:
import os
import time
import copy
import json
from PIL import Image

In [103]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [104]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split

In [105]:
from transformers import ViTConfig, ViTModel, ViTImageProcessor

## Preparing Data Loaders

In [106]:
class SpoofDataset(Dataset):
    def __init__(self, data_dir, label_file, transform=None):
        """
        Args:
            data_dir: Directory with all the images, separated into 'live' and 'spoof' subdirectories.
            label_file: Path to JSON file with labels, where label 0 indicates 'live' and others indicate 'spoof'.
            transform: Optional transform.
        """
        self.data_dir = data_dir
        self.transform = transform
        with open(label_file, 'r') as f:
            self.labels = json.load(f)

        self.img_paths = []
        self.img_labels = []
        self.features_list = []
        for img_name, features in self.labels.items():            
            # final item in feature is for live vs spoof
            label = features[-1]
            subfolder = 'live' if label == 0 else 'spoof'
            full_path = os.path.join(self.data_dir, subfolder, img_name)
            
            if os.path.exists(full_path):
                self.img_paths.append(full_path)
                self.img_labels.append(label)
                self.features_list.append(features[:-1])

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
            
        label = self.img_labels[idx]
        features = self.features_list[idx]

        return image, label, features


In [107]:
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.ToPILImage()
])

In [108]:
data_dir = "/Users/jasminecjwchen/Documents/GitHub/COMS-4995-ACV-Project/preprocessed_data"
test_dir = "/Users/jasminecjwchen/Documents/GitHub/COMS-4995-ACV-Project/unseen_data"
output_dir = '/Users/jasminecjwchen/Documents/GitHub/COMS-4995-ACV-Project/split_data'

In [109]:
label_file = "/Users/jasminecjwchen/Documents/GitHub/COMS-4995-ACV-Project/preprocessing/test_labels v2.json"
dataset = SpoofDataset(data_dir, label_file, data_transforms)

In [110]:
# splitting
num_train = int(len(dataset) * 0.8)
num_val = len(dataset) - num_train
train_dataset, val_dataset = random_split(dataset, [num_train, num_val])

In [111]:
def collate_fn(batch):
    # Filter failed images first
    batch = list(filter(lambda x: x is not None, batch))

    images = [sample[0] for sample in batch]
    labels = torch.LongTensor([sample[1] for sample in batch])
    features = torch.stack([torch.Tensor(sample[2]) for sample in batch])
    
    return images, labels, features

In [112]:
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True, collate_fn=collate_fn),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=False, pin_memory=True, collate_fn=collate_fn)
}

In [113]:
dataset_sizes = {
    "train": len(train_dataset),
    "val": len(val_dataset)
}

In [118]:
print(dataset_sizes)

{'train': 31876, 'val': 7970}


## VIT

In [119]:
class VITModel(nn.Module):
    # vit config params are passed through kwargs
    def __init__(self, freeze_vit = True):
        super().__init__()
        
        self.tokenizer = ViTImageProcessor()
        
        self.model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
        self.logistic = nn.Linear(151296, 5)
        
        if freeze_vit: 
            for param in self.model.parameters():
                param.requires_grad = False
    
    def forward(self, image):
        encoded_image = self.tokenizer(image, return_tensors = "pt")
        model_output = self.model(**encoded_image)
        
        flattened_output = model_output.last_hidden_state.flatten(start_dim = 1)
        return self.logistic(flattened_output)

## Training

In [120]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")
use_gpu = (device != "cpu")

Using cpu


In [121]:
def train_model(model, optimizer, criterion = None, scheduler = None, dataloaders = dataloaders, num_epochs = 1, patience = 10, output_filename = "best_model.pth"):    
    since = time.time()
    
    # Initialize best metrics tracking
    best_metrics = {
        'epoch': 0,
        'val_loss': float('inf'),
        'val_accuracy': 0,
        'val_precision': 0,
        'val_recall': 0,
        'val_f1': 0,
    }
    
    best_model_wts = copy.deepcopy(model.state_dict())
    
    if not criterion:
        criterion = nn.CrossEntropyLoss()
    
    # use patience for early stopping when validation isnt getting better
    patience_left = patience

    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            all_preds = []
            all_labels = []
            
            for image, labels, _ in dataloaders[phase]:
                if use_gpu:
                    labels = labels.to(device)
                
                optimizer.zero_grad()
                
                # forward
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(image)
                    preds = torch.argmax(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * len(image)
                #print(running_loss, running_corrects)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_accuracy = accuracy_score(all_labels, all_preds)
            epoch_precision = precision_score(all_labels, all_preds, zero_division=0, average='macro')
            epoch_recall = recall_score(all_labels, all_preds, zero_division=0, average='macro')
            epoch_f1 = f1_score(all_labels, all_preds, zero_division=0, average='macro')

            epoch_time = time.time() - epoch_start_time

            # deep copy the model if it's best so far
            if phase == "val" and (epoch_loss < best_metrics['val_loss'] or epoch_recall > best_metrics['val_recall']):
                best_metrics.update({
                    'epoch': epoch + 1,
                    'val_loss': epoch_loss,
                    'val_accuracy': epoch_accuracy,
                    'val_precision': epoch_precision,
                    'val_recall': epoch_recall,
                    'val_f1': epoch_f1,
                })
                
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), output_filename)
                patience_left = patience
            elif phase == "val":
                patience_left -= 1
            
            print(f'Epoch {epoch}/{num_epochs - 1} {phase} complete in {epoch_time:.4f} seconds. {phase} loss: {epoch_loss:.4f} recall: {epoch_recall:.4f}. Patience left: {patience_left}')
            
        if patience_left <= 0:
            print("Ran out of patience. Stopping early")
            break
        
        if scheduler:
            scheduler.step()
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f"Best Metrics at Epoch {best_metrics['epoch']}:")
    for metric, value in best_metrics.items():
        if metric != 'epoch':
            print(f"{metric.capitalize()}: {value:.4f}")

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [122]:
model = VITModel()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [124]:
train_model(model, optimizer, output_filename = "vit_multiclass.pth", num_epochs = 10, patience = 2)

Epoch 0/9 train complete in 6282.5754 seconds. train loss: 0.2800 recall: 0.9319. Patience left: 2
Epoch 0/9 val complete in 7108.8765 seconds. val loss: 0.7141 recall: 0.8774. Patience left: 2
Epoch 1/9 train complete in 7868.9059 seconds. train loss: 0.1768 recall: 0.9575. Patience left: 2
Epoch 1/9 val complete in 8785.2925 seconds. val loss: 0.5414 recall: 0.9079. Patience left: 2
Epoch 2/9 train complete in 14103.3475 seconds. train loss: 0.1485 recall: 0.9663. Patience left: 2
Epoch 2/9 val complete in 14898.3742 seconds. val loss: 0.7063 recall: 0.8993. Patience left: 1
Epoch 3/9 train complete in 4149.0135 seconds. train loss: 0.1335 recall: 0.9731. Patience left: 1
Epoch 3/9 val complete in 4968.5583 seconds. val loss: 0.6531 recall: 0.9141. Patience left: 2
Epoch 4/9 train complete in 3397.9292 seconds. train loss: 0.1469 recall: 0.9741. Patience left: 2
Epoch 4/9 val complete in 4213.7454 seconds. val loss: 0.9924 recall: 0.8920. Patience left: 1
Epoch 5/9 train complete in 

VITModel(
  (model): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_feature